In [1]:
import re
import pandas as pd
from collections import Counter

## 1. Finding the Unique Words

In [2]:
with open('textCorpus.txt','r') as fd:
    lines = fd.readlines()
    words = []
    for line in lines:
        words += re.findall('\w+',line.lower())
        
print("All the words: ",len(words))
vocab = list(set(words))
print("Unique words: ",len(vocab))

All the words:  1115585
Unique words:  32198


## 2. Finding the Probability Distribution

In [3]:
word_probability = dict(Counter(words))

for i in word_probability:
    word_probability[i]=word_probability[i]/len(words)

## 3. Text Preprocessing
### Splitting

In [4]:
def split(word):  
    parts = []
    for i in range(len(word) + 1):
        parts += [(word[ : i], word[i : ])]
    return parts

### 3.1) Delete

'loave' -> 'love'

In [5]:
def delete(word):
    
    output = []
    for l,r in split(word):
        output.append(l + r[1:])
    return output

delete('loave')

['oave', 'lave', 'love', 'loae', 'loav', 'loave']

### 3.2) Swap

'lvoe' -> 'love'

In [6]:
def swap(word):
        
    output = []    
    for l,r in split(word):
        if (len(r) > 1):
            output.append(l + r[1] + r[0] + r[2:])
    return output
            
swap('lvoe')

['vloe', 'love', 'lveo']

### 3.3) Replace

'lave' -> 'love'

In [7]:
def replace(word):
    
    characters = 'abcdefghijklmnopqrstuvwxyz'
    output = []    

    for l,r in split(word):
        for char in characters:
            output.append(l + char +  r[1:])
    return output

len(replace('lave'))

130

### 3.4) Insert

'lve' -> 'love'

In [8]:
def insert(word):

    characters = 'abcdefghijklmnopqrstuvwxyz'
    output = []

    for l,r in split(word):
        for char in characters:
            output.append(l + char + r)

    return output

len(insert('lve'))

104

## 4. Finding the Prediction (Level - 1)
### 4.1) Combining Possible Words

In [9]:
def edit(word):   
    return list(set(insert(word) + delete(word) + swap(word) + replace(word)))

### 4.2) Predicting the Word

In [10]:
def spell_check_edit_1(word, count = 5):
    
    output = []
    suggested_words = edit(word)
    
    for wrd in suggested_words:        
        if wrd in word_probability.keys():
            output.append([wrd, word_probability[wrd]])
            
    return list(pd.DataFrame(output, columns = ['word','prob']).sort_values(by = 'prob', ascending = False).head(count)['word'].values)

In [11]:
spell_check_edit_1('famely')

['family', 'namely', 'lamely']

## 5. Finding the Prediction (Level - 2)
### 5.1) Combining Possible Words

In [12]:
def spell_check_edit_2(word, count = 5):
    
    output = []
    suggested_words = edit(word)       # Level one Edit
    
    for e1 in edit(word):
        suggested_words += edit(e1)    # Second Level Edit 
    
    suggested_words = list(set(suggested_words))
    
    for wrd in suggested_words:
        if wrd in word_probability.keys():
            output.append([wrd, word_probability[wrd]])
    return list(pd.DataFrame(output, columns = ['word','prob']).sort_values(by = 'prob', ascending = False).head(count)['word'].values)
        
spell_check_edit_2('fameli')

['family', 'namely', 'fame', 'amelie', 'camel']